# Try OpenEO API

- 11/02/2025: get timeout when using download() (need to check again)
- 12/02/2025: download() works normally -> it took around 2-3 mins to download the image (4 bands), around 4-5 mins (all bands)

In [69]:
import openeo

In [70]:
connection = openeo.connect("openeo.dataspace.copernicus.eu")
# connection = openeo.connect("openeofed.dataspace.copernicus.eu")

In [71]:
connection

<Connection to 'https://openeo.dataspace.copernicus.eu/openeo/1.2/' with NullAuth>

In [72]:
connection.authenticate_oidc()

Authenticated using refresh token.


<Connection to 'https://openeo.dataspace.copernicus.eu/openeo/1.2/' with OidcBearerAuth>

bbox of Altstadt, Dresden, Germany
- [13.6864402, 51.0280799, 13.7872926, 51.0766681] (minx, miny, maxx, maxy)

bbox of Dresden, Germany
- [13.5793237, 50.974937 , 13.9660626, 51.1777202] 

In [73]:
sentinel2_cube = connection.load_collection(
    "SENTINEL2_L2A",
    spatial_extent={"west": 13.5793237, "south": 50.974937, "east": 13.9660626, "north": 51.1777202},
    temporal_extent=["2024-05-01", "2024-08-31"],
    max_cloud_cover=5
)

# sentinel2_cube = connection.load_collection(
#     "SENTINEL2_L2A",
#     spatial_extent={"west": 13.6864402, "south": 51.0280799, "east": 13.7872926, "north": 51.0766681},
#     temporal_extent=["2024-05-01", "2024-08-31"],
#     max_cloud_cover=5,
#     bands=["B02", "B03", "B04", "B08", "B11"]
# )

In [74]:
sentinel2_cube.print_json()

{
  "process_graph": {
    "loadcollection1": {
      "process_id": "load_collection",
      "arguments": {
        "id": "SENTINEL2_L2A",
        "properties": {
          "eo:cloud_cover": {
            "process_graph": {
              "lte1": {
                "process_id": "lte",
                "arguments": {
                  "x": {
                    "from_parameter": "value"
                  },
                  "y": 5
                },
                "result": true
              }
            }
          }
        },
        "spatial_extent": {
          "west": 13.5793237,
          "south": 50.974937,
          "east": 13.9660626,
          "north": 51.1777202
        },
        "temporal_extent": [
          "2024-05-01",
          "2024-08-31"
        ]
      },
      "result": true
    }
  }
}


In [75]:
sentinel2_cube_latest = sentinel2_cube.max_time()

In [76]:
sentinel2_cube_latest.to_json()

'{\n  "process_graph": {\n    "loadcollection1": {\n      "process_id": "load_collection",\n      "arguments": {\n        "id": "SENTINEL2_L2A",\n        "properties": {\n          "eo:cloud_cover": {\n            "process_graph": {\n              "lte1": {\n                "process_id": "lte",\n                "arguments": {\n                  "x": {\n                    "from_parameter": "value"\n                  },\n                  "y": 5\n                },\n                "result": true\n              }\n            }\n          }\n        },\n        "spatial_extent": {\n          "west": 13.5793237,\n          "south": 50.974937,\n          "east": 13.9660626,\n          "north": 51.1777202\n        },\n        "temporal_extent": [\n          "2024-05-01",\n          "2024-08-31"\n        ]\n      }\n    },\n    "reducedimension1": {\n      "process_id": "reduce_dimension",\n      "arguments": {\n        "data": {\n          "from_node": "loadcollection1"\n        },\n      

In [77]:
sentinel2_cube_med = sentinel2_cube.reduce_dimension(dimension='t', reducer='median')

In [78]:
# sentinel2_cube_latest.download("../data/raw/sentinel-2/sentinel2_openeo.geotiff", format="GTiff")
sentinel2_cube_latest.download("../data/raw/sentinel-2/sentinel2_openeo_dresden.geotiff", format="GTiff")

In [79]:
# sentinel2_cube_med.download("../data/raw/sentinel-2/sentinel2_openeo_med_3months.geotiff", format="GTiff")
# sentinel2_cube_med.download("../data/raw/sentinel-2/sentinel2_openeo_med_3months_dresden.geotiff", format="GTiff")
sentinel2_cube_med.download("../data/raw/sentinel-2/sentinel2_openeo_med_4months_dresden.geotiff", format="GTiff")

# Try with STAC

In [ ]:
from pystac_client import Client

In [ ]:
api_url = "https://earth-search.aws.element84.com/v1"

In [ ]:
client = Client.open(api_url)

In [ ]:
collections = client.get_collections()

In [ ]:
for collection in collections:
    print(collection)

In [ ]:
s2_search = client.search(
    collections=["sentinel-2-l2a"],
    bbox=[13.6864402, 51.0280799, 13.7872926, 51.0766681],
    query=["eo:cloud_cover<10"],
    sortby=["-properties.eo:cloud_cover"],
    datetime="2024-05-01/2024-08-31"
)

In [ ]:
s2_items = s2_search.item_collection()
for item in s2_items:
    print(item.properties["eo:cloud_cover"])

In [ ]:
selected_item = s2_items[7]

In [ ]:
assets = selected_item.assets
print(assets.keys())

In [ ]:
for key, asset in assets.items():
    print(f"{key}:{asset.title}")

In [ ]:
print(assets["thumbnail"].href)

In [ ]:
import rioxarray
import matplotlib.pyplot as plt

In [ ]:
nir_href = assets["nir"].href
nir = rioxarray.open_rasterio(nir_href)
nir

In [ ]:
nir.rio.crs

In [ ]:
nir_reprojected = nir.rio.reproject("EPSG:4326")
print(nir_reprojected.rio.crs)

In [ ]:
city_bbox = [13.6864402, 51.0280799, 13.7872926, 51.0766681]
raster_clip_box = nir_reprojected.rio.clip_box(*city_bbox)
print(raster_clip_box.shape)

In [ ]:
%%timeit -n 5 -r 1
plt.imshow(raster_clip_box[0, :, :], cmap='gray')
plt.clim(vmin=10, vmax=5000)

# Try GEE

In [ ]:
import ee

ee.Authenticate()

In [ ]:
ee.Initialize(project='ee-japanpitchaporn')

In [ ]:
bbox = ee.Geometry.BBox(13.6864402, 51.0280799, 13.7872926, 51.0766681)

In [ ]:
s2_collection = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
                 .filterDate('2024-05-01', '2024-08-31')
                 .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 5))
                 .filter(ee.Filter.bounds(bbox))
                )

In [ ]:
s2_collection.getInfo()

In [ ]:
med_s2_img = s2_collection.median().clip(bbox)

In [ ]:
import geemap

In [ ]:
geemap.ee_initialize()

In [ ]:
Map = geemap.Map()

In [ ]:
vis = {
    'min': 0.0,
    'max': 3000,
    'bands': ['B4', 'B3', 'B2'],
}

Map.setCenter(13.6864402, 51.0280799, 12)
Map.addLayer(med_s2_img, vis, 'Sentinel-2')
Map.addLayer(bbox, {}, 'Polygon')
Map

In [ ]:
# Retrieve the projection information from a band of the original image.
# Call getInfo() on the projection to request a client-side object containing
# the crs and transform information needed for the client-side Export function.
# projection = med_s2_img.select('B2').projection().getInfo()

In [ ]:
med_s2_img = med_s2_img.setDefaultProjection('EPSG:32633')

In [ ]:
# geemap.download_ee_image(med_s2_img, filename='test.tif', crs='EPSG:32633', region=bbox)
geemap.download_ee_image(med_s2_img, filename='test.tif', crs='EPSG:32633')

# Try earthaccess

In [ ]:
import json

from odc.stac import configure_rio, load
from pystac_client import Client as PystacClient
import geopandas as gpd
import numpy as np

from utils import hls_config

In [ ]:
# from dask.distributed import Client as DaskClient

# dask_client = DaskClient()
# dask_client

In [ ]:
# The catalog URL for the Earthdata STAC containing HLS
catalog = "https://cmr.earthdata.nasa.gov/cloudstac/LPCLOUD/"

# pystac_client is used to connect to the catalog
stac_client = PystacClient.open(catalog)

In [ ]:
# Specify both Landsat and Sentinel-2 HLS collections for the query
collections = ["HLSS30.v2.0"]

In [ ]:
# Set the start date and end date
start_date = "2024-05-01"
end_date = "2024-08-31"

# Format the date range for the query
date_range = f"{start_date}/{end_date}"

In [ ]:
# Search for items in the collection
items = list(
    stac_client.search(
        collections=collections, bbox=[13.6864402, 51.0280799, 13.7872926, 51.0766681], datetime=date_range
    ).items()
)

print(f"Found {len(items)} items")

In [ ]:
import earthaccess

earthaccess.login()

In [ ]:
# This code will be wrapped by `get_edl_token` in the next earthaccess release
token = earthaccess.__auth__.token["access_token"]

# Configure rasterio to use cloud defaults, and GDAL to use the authorization token
header_string = f"Authorization: Bearer {token}"
configure_rio(cloud_defaults=True, GDAL_HTTP_HEADERS=header_string)

In [ ]:
data = load(
    items,
    resolution=30,
    crs="EPSG:32633",
    chunks={},
    groupby="solar_day",
    # stac_cfg=hls_config,
    bands=["B02", "B03", "B04", "B08", "B11"],
    bbox=[13.6864402, 51.0280799, 13.7872926, 51.0766681]
)

data

In [ ]:
from osgeo import gdal
# GDAL configurations used to successfully access LP DAAC Cloud Assets via vsicurl 
gdal.SetConfigOption('GDAL_HTTP_COOKIEFILE','~/cookies.txt')
gdal.SetConfigOption('GDAL_HTTP_COOKIEJAR', '~/cookies.txt')
gdal.SetConfigOption('GDAL_DISABLE_READDIR_ON_OPEN','EMPTY_DIR')
gdal.SetConfigOption('CPL_VSIL_CURL_ALLOWED_EXTENSIONS','TIF')
gdal.SetConfigOption('GDAL_HTTP_UNSAFESSL', 'YES')
gdal.SetConfigOption('GDAL_HTTP_MAX_RETRY', '10')
gdal.SetConfigOption('GDAL_HTTP_RETRY_DELAY', '0.5')

In [ ]:
# # Define a simple plotting function to reuse
# def plot_rgb(data):
#     # Select the red green and blue bands
#     rgb = data[["B04", "B03", "B02"]]
#     print(rgb)
#     # Select a subset of images that show clear and cloudy images
#     rgb_subset = rgb.isel(time=slice(4, 7)).to_array()
#     print(rgb_subset)
    
#     # Display the image
#     rgb_subset.plot.imshow()


# plot_rgb(data)